In [1]:
import numpy as np
import os
from svm import SVM
import glob
from sklearn.model_selection import train_test_split
import open3d as o3d
from sklearn.decomposition import PCA



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
class1_path = r"feature_extracted_npy_sorted\5"
class2_path = r"feature_extracted_npy_sorted\2"


In [3]:
def process_npy_files(file_path, label):
    npy_files = glob.glob(os.path.join(file_path, "*.npy"))
    data_list = []
    labels = []
    
    for npy_file in npy_files:
        # Load NPY file directly
        feature_vector = np.load(npy_file)
        data_list.append(feature_vector)
        labels.append(label)
    
    return np.array(data_list), np.array(labels)

# Process both classes with standardized number of points
class1_data, class1_labels = process_npy_files(class1_path, 1)
class2_data, class2_labels = process_npy_files(class2_path, -1)

# Combine data from both classes
X = np.vstack([class1_data, class2_data])
y = np.array(list(class1_labels) + list(class2_labels))
random_indices = np.random.permutation(len(X))
X = X[random_indices]
y = y[random_indices]

In [4]:
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (1006, 1024)
y shape: (1006,)


In [5]:
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train SVM model
svm_model = SVM(kernel_type='rbf', sigma=0.1)
svm_model.pso_settings(pso_max_iter=10, 
                       pso_n_particles=20, 
                       pso_w=0.7, pso_c1=2.05, 
                       pso_c2=2.05, 
                       admm_max_iter=10)
svm_model.train(X_train, y_train)

# Calculate and print training accuracy
train_accuracy = svm_model.score(X_train, y_train)
print(f"Training accuracy: {train_accuracy:.4f}")

# Calculate and print test accuracy
test_accuracy = svm_model.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

PSO_train called with max_iter=10, n_particles=20, w=0.7, c1=2.05, c2=2.05, admm_max_iter=10
Iteration 1/10, Best fitness: 0.8063
Best parameters: rho=1.6771, sigma=0.3922, C=6.2315
Iteration 2/10, Best fitness: 0.8250
Best parameters: rho=1.6678, sigma=0.2170, C=6.5836
Iteration 3/10, Best fitness: 0.8250
Best parameters: rho=1.6678, sigma=0.2170, C=6.5836
Iteration 4/10, Best fitness: 0.8438
Best parameters: rho=1.8042, sigma=0.3342, C=6.2926
Iteration 5/10, Best fitness: 0.8438
Best parameters: rho=1.8042, sigma=0.3342, C=6.2926
Iteration 6/10, Best fitness: 0.8438
Best parameters: rho=1.8042, sigma=0.3342, C=6.2926
Iteration 7/10, Best fitness: 0.8438
Best parameters: rho=1.8042, sigma=0.3342, C=6.2926
Iteration 8/10, Best fitness: 0.8500
Best parameters: rho=1.6558, sigma=0.3078, C=6.6243
Iteration 9/10, Best fitness: 0.8500
Best parameters: rho=1.6558, sigma=0.3078, C=6.6243
Iteration 10/10, Best fitness: 0.8500
Best parameters: rho=1.6558, sigma=0.3078, C=6.6243
Training accurac

In [6]:
model_save_path = "model_bin_83.pkl"
svm_model.save_model(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to model_bin_83.pkl
